In [1]:
import chess
import os
import time
import chess.svg
import cairosvg
import random

# Define piece values for material evaluation
picesCost = {
    "n": 0,
    "p": 1,
    "b": 3,
    "k": 3,
    "r": 5,
    "q": 9
}

class Heuristics:
    @staticmethod
    def numberOfMoves(boardState: chess.Board, color):
        """
        Calculate the number of legal moves available for a specific color.

        Args:
            boardState (chess.Board): The chessboard state.
            color (str): The color for which to calculate legal moves ("white" or "black").

        Returns:
            int: The number of legal moves.
        """
        return boardState.legal_moves.count()

    @staticmethod
    def materialH(boardState: chess.Board, color: str):
        """
        Calculate the material advantage for a given color in a chessboard state.

        Args:
            boardState (chess.Board): The chessboard state.
            color (str): The color for which to calculate material advantage ("white" or "black").

        Returns:
            int: The material advantage score.
        """
        pieces = boardState.board_fen()
        if color == "white":
            return (sum(picesCost[piece.lower()] for piece in pieces if piece.isupper() and piece.isalpha())) - (
                    sum(picesCost[piece.lower()] for piece in pieces if piece.islower() and piece.isalpha()))
        return (sum(picesCost[piece.lower()] for piece in pieces if piece.islower() and piece.isalpha())) - (
                sum(picesCost[piece.lower()] for piece in pieces if piece.isupper() and piece.isalpha()))

    @staticmethod
    def numberPieces(boardState: chess.Board, color: str):
        """
        Calculate the difference in the number of pieces for a given color compared to the opponent's color.

        Args:
            boardState (chess.Board): The chessboard state.
            color (str): The color for which to calculate the difference in the number of pieces ("white" or "black").

        Returns:
            int: The difference in the number of pieces.
        """
        pieces = boardState.board_fen()
        if color == "white":
            return (sum(1 for piece in pieces if piece.isupper() and piece.isalpha())) - (
                    sum(1 for piece in pieces if piece.islower() and piece.isalpha()))
        return (sum(1 for piece in pieces if piece.islower() and piece.isalpha())) - (
                sum(1 for piece in pieces if piece.isupper and piece.isalpha()))

class Agent:
    def __init__(self, heuristic, color, max_depth=2, save_folder="Images"):
        """
        Initialize an Agent with the specified heuristic, color, and other parameters.

        Args:
            heuristic (callable): The heuristic function used for evaluation.
            color (str): The color of the agent ("white" or "black").
            max_depth (int): The maximum search depth for the minimax algorithm.
            save_folder (str): The folder to save board state images.
        """
        self.heuristic = heuristic
        self.color = color
        self.max_depth = max_depth
        self.save_folder = save_folder
        self.move_counter = 1

    def minimax(self, board, depth, maximizing_player):
        """
        Implement the minimax algorithm to evaluate board states and determine the best move.

        Args:
            board (chess.Board): The current chessboard state.
            depth (int): The current search depth in the tree.
            maximizing_player (bool): True if the current player is maximizing, False if minimizing.

        Returns:
            int: The evaluation score for the board state.
        """
        if depth == 0 or board.is_game_over():
            return self.heuristic(board, self.color)

        if maximizing_player:
            max_eval = float('-inf')
            for move in board.legal_moves:
                new_board = board.copy()
                new_board.push_uci(str(move))
                eval = self.minimax(new_board, depth - 1, False)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in board.legal_moves:
                new_board = board.copy()
                new_board.push_uci(str(move))
                eval = self.minimax(new_board, depth - 1, True)
                min_eval = min(min_eval, eval)
            return min_eval

    def makeMove(self, boardState):
        """
        Select the best move for the agent using the minimax algorithm with alpha-beta pruning.

        Args:
            boardState (chess.Board): The current chessboard state.

        Returns:
            str: The UCI notation of the selected move.
        """
        best_moves = []
        best_eval = float('-inf')
        for move in boardState.legal_moves:
            new_board = boardState.copy()
            new_board.push_uci(str(move))
            eval = self.minimax(new_board, self.max_depth, False)
            if eval > best_eval:
                best_eval = eval
                best_moves = [move]
            elif eval == best_eval:
                best_moves.append(move)

        # Seed the random number generator with a unique value for choosing one global min (if there is more than one)
        random.seed()
        best_move = random.choice(best_moves)

        move_uci = boardState.san(best_move)
        #self.save_board_state_image(boardState, self.move_counter)
        self.move_counter += 1

        return move_uci

    def save_board_state_image(self, board, step):
        """
        Generate and save an image of the current chessboard state.

        Args:
            board (chess.Board): The current chessboard state.
            step (int): The step number to create a unique image file.
        """
        png_path = os.path.join(self.save_folder, f'step_{step}.png')
        svg_content = chess.svg.board(board=board)
        with open(png_path, 'wb') as png_file:
            cairosvg.svg2png(bytestring=svg_content, write_to=png_file)

def Chess(agent1, agent2):
    """
    Simulate a chess game between two agents.

    Args:
        agent1 (Agent): The first agent.
        agent2 (Agent): The second agent.
    """
    startTime = time.time()
    board = chess.Board()

    while not (board.is_checkmate() or board.is_fifty_moves() or board.is_insufficient_material()
               or board.is_stalemate() or board.is_fivefold_repetition()):
        if board.turn:
            move = agent1.makeMove(board)
            board.push_san(move)
        else:
            move = agent2.makeMove(board)
            board.push_san(move)

    if board.is_fifty_moves():
        print("55 moves gained, it's a draw")
    if board.is_fivefold_repetition():
        print("Fivefold repetition, it's a draw")
    elif board.is_checkmate():
        if board.turn:
            print("White is under checkmate, black wins")
        else:
            print("Black is under checkmate, white wins")
    if board.is_insufficient_material():
        print("Insufficient material, it's a draw")

    print("Number of moves for Agent 1:", len(board.move_stack) // 2)
    print("Number of moves for Agent 2:", len(board.move_stack) // 2)
    print("--- %s seconds ---" % round((time.time() - startTime), 2))

In [2]:
agent1 = Agent(Heuristics.numberOfMoves, "white")
agent2 = Agent(Heuristics.numberOfMoves, "black")
Chess(agent1, agent2)

Fivefold repetition, it's a draw
Number of moves for Agent 1: 25
Number of moves for Agent 2: 26
--- 75.0 seconds ---


In [3]:
agent1 = Agent(Heuristics.materialH, "white")
agent2 = Agent(Heuristics.materialH, "black")
Chess(agent1, agent2)

Fivefold repetition, it's a draw
Number of moves for Agent 1: 55
Number of moves for Agent 2: 56
--- 340.0 seconds ---


In [4]:
 agent1 = Agent(Heuristics.numberPieces, "white")
agent2 = Agent(Heuristics.numberPieces, "black")
Chess(agent1, agent2)

Black is under checkmate, white wins
Number of moves for Agent 1: 35
Number of moves for Agent 2: 35
--- 247.0 seconds ---
